In [61]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import itertools
import scipy.misc
from IPython import display
import time
%matplotlib inline

In [62]:
import numpy as np
import random
import itertools
import scipy.misc
import matplotlib.pyplot as plt


class gameOb():
    def __init__(self,coordinates,size,intensity,channel,reward,name):
        self.x = coordinates[0]
        self.y = coordinates[1]
        self.size = size
        self.intensity = intensity
        self.channel = channel
        self.reward = reward
        self.name = name
        
class gameEnv():
    def __init__(self,partial,size):
        self.sizeX = size
        self.sizeY = size
        self.actions = 4
        self.objects = []
        self.partial = partial
        self.a = self.reset()
#         plt.imshow(self.a,interpolation="nearest")
        
        
    def reset(self):
        self.objects = []
        hero = gameOb(self.newPosition(),1,1,2,None,'hero')
        self.objects.append(hero)
        bug = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug)
        hole = gameOb(self.newPosition(),1,1,0,-1,'fire')
        self.objects.append(hole)
        bug2 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug2)
        hole2 = gameOb(self.newPosition(),1,1,0,-1,'fire')
        self.objects.append(hole2)
        bug3 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug3)
        bug4 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug4)
        state = self.renderEnv()
        self.state = state
        return state

    def moveChar(self,direction):
        # 0 - up, 1 - down, 2 - left, 3 - right
        hero = self.objects[0]
        heroX = hero.x
        heroY = hero.y
        penalize = 0.05
        if direction == 0 and hero.y >= 1:
            hero.y -= 1
        if direction == 1 and hero.y <= self.sizeY-2:
            hero.y += 1
        if direction == 2 and hero.x >= 1:
            hero.x -= 1
        if direction == 3 and hero.x <= self.sizeX-2:
            hero.x += 1     
        if hero.x == heroX and hero.y == heroY:
            penalize = .2
        self.objects[0] = hero
        return penalize
    
    def newPosition(self):
        iterables = [ range(self.sizeX), range(self.sizeY)]
        points = []
        for t in itertools.product(*iterables):
            points.append(t)
        currentPositions = []
        for objectA in self.objects:
            if (objectA.x,objectA.y) not in currentPositions:
                currentPositions.append((objectA.x,objectA.y))
        for pos in currentPositions:
            points.remove(pos)
        location = np.random.choice(range(len(points)),replace=False)
        return points[location]

    def checkGoal(self):
        others = []
        for obj in self.objects:
            if obj.name == 'hero':
                hero = obj
            else:
                others.append(obj)
        ended = False
        for other in others:
            if hero.x == other.x and hero.y == other.y:
                self.objects.remove(other)
                if other.reward == 1:
                    self.objects.append(gameOb(self.newPosition(),1,1,1,1,'goal'))
                else: 
                    self.objects.append(gameOb(self.newPosition(),1,1,0,-1,'fire'))
                return other.reward,True
        if ended == False:
            return 0.0,False

    def renderEnv(self):
        #a = np.zeros([self.sizeY,self.sizeX,3])
        self.a = np.ones([self.sizeY+2,self.sizeX+2,3])
        self.a[1:-1,1:-1,:] = 0
        hero = None
        for item in self.objects:
            self.a[item.y+1:item.y+item.size+1,item.x+1:item.x+item.size+1,item.channel] = item.intensity
            if item.name == 'hero':
                hero = item
        if self.partial == True:
            self.a = self.a[hero.y:hero.y+3,hero.x:hero.x+3,:]
        b = scipy.misc.imresize(self.a[:,:,0],[84,84,1],interp='nearest')
        c = scipy.misc.imresize(self.a[:,:,1],[84,84,1],interp='nearest')
        d = scipy.misc.imresize(self.a[:,:,2],[84,84,1],interp='nearest')
        self.a = np.stack([b,c,d],axis=2)
        return self.a

    def step(self,action):
        penalty = self.moveChar(action)
        reward,done = self.checkGoal()
        state = self.renderEnv()
        return state,(reward-penalty),done

In [63]:
env = gameEnv(partial=False,size=5)

In [64]:
class Qnetwork():
    def __init__(self, h_size, frameShape, batch_size):
        # The network recieves a frame from the game, flattened into an array.
        # It then resizes it and processes it through four convolutional layers.
        
        # raw pixel data (grayscale, so only 1 channel)
        self.rgb_array = tf.placeholder(shape=[None, frameShape[0], frameShape[1], 3], dtype=tf.float32)
        
        # tf input: a 4-D tensor [batch_size, height, width, channels]
        self.imageIn = tf.image.resize_images(self.rgb_array, 84, 84, 3)
        
        self.conv1 = tf.contrib.layers.convolution2d(
            inputs = self.imageIn,
            num_outputs = 32,
            kernel_size = [8, 8],
            stride = [4, 4],
            padding = 'VALID',
            biases_initializer = None)
        
        self.conv2 = tf.contrib.layers.convolution2d(
            inputs = self.conv1,
            num_outputs = 64,
            kernel_size = [4, 4],
            stride = [2, 2],
            padding = 'VALID',
            biases_initializer = None)
        
        self.conv3 = tf.contrib.layers.convolution2d(
            inputs = self.conv2,
            num_outputs = 64,
            kernel_size = [3, 3],
            stride = [1, 1],
            padding = 'VALID',
            biases_initializer = None)
        
        self.conv4 = tf.contrib.layers.convolution2d(
            inputs = self.conv3,
            num_outputs = 512,
            kernel_size = [7, 7],
            stride = [1, 1],
            padding = 'VALID',
            biases_initializer = None)
        
        # We take the output from the final convolutional layer and split it
        # into separate advantage and value streams.
        
        # split on the 3rd dimension into 2 different parts
        self.streamAC, self.streamVC = tf.split(3, 2, self.conv4)
        
        # flatten to [batch_size, k]
        self.streamA = tf.contrib.layers.flatten(self.streamAC)
        self.streamV = tf.contrib.layers.flatten(self.streamVC)
        
        # weights for advantage and value stream layer
        self.AW = tf.Variable(tf.random_normal([int(h_size/2) , env.actions]))
        self.VW = tf.Variable(tf.random_normal([int(h_size/2) , 1]))
        
        # output of advantage and value layer
        self.Advantage = tf.matmul(self.streamA, self.AW)
        self.Value = tf.matmul(self.streamV, self.VW)
        
        # Then combine them together to get our final Q-values.
        # Q(s, a) = V(s) + A(a)
        self.Qout = self.Value + tf.sub(
            self.Advantage,
            tf.reduce_mean( # TODO: understand how this A(a) is calculated
                self.Advantage,
                reduction_indices=1,
                keep_dims=True))
        
        # index of max value across 1st dimension
        self.predict = tf.argmax(self.Qout, 1)
        
        # Below we obtain the loss by taking the sum of squares difference between
        # the target and prediction Q values.
        
        # target Q value
        self.targetQ = tf.placeholder(shape=[None], dtype=tf.float32)
        
        # possible actions
        self.actions = tf.placeholder(shape=[None], dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions, env.actions, dtype=tf.float32)
        
        # predicted Q values
        self.Q = tf.reduce_sum(tf.mul(self.Qout, self.actions_onehot), reduction_indices=1)
        
        # error = sum( (target - actual)^2 ) / batch_size
        self.td_error = tf.square(self.targetQ - self.Q)
        self.loss = tf.reduce_mean(self.td_error)
        
        # define trainer
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        
        # define optimizer
        self.updateModel = self.trainer.minimize(self.loss)

In [65]:
class experience_buffer():
    """Used to store experiences and samples randomly to train the network."""
    def __init__(self, buffer_size=50000):
        self.buffer_size = buffer_size
        # state, action, reward, state1, done
#         self.states = np.array([])
#         self.actions = np.array([])
#         self.rewards = np.array([])
#         self.states_ = np.array([])
#         self.dones = np.array([])
        self.states = []
        self.actions = []
        self.rewards = []
        self.states_ = []
        self.dones = []
        
    def add(self, states, actions, rewards, states_, dones):
        if len(self.states) + len(states) >= self.buffer_size:
            self.states = self.states[:(self.buffer_size - len(states))]
            self.actions = self.actions[:(self.buffer_size - len(actions))]
            self.rewards = self.rewards[:(self.buffer_size - len(rewards))]
            self.states_ = self.states_[:(self.buffer_size - len(states_))]
            self.dones = self.dones[:(self.buffer_size - len(dones))]

        self.states.extend(states)
        self.actions.extend(actions)
        self.rewards.extend(rewards)
        self.states_.extend(states_)
        self.dones.extend(dones)
#         self.states = np.append(self.states, states)
#         self.actions = np.append(self.actions, actions)
#         self.rewards = np.append(self.rewards, rewards)
#         self.states_ = np.append(self.states_, states_)
#         self.dones = np.append(self.dones, dones)
        
    def sample(self, size):
        samples = random.sample(range(len(self.actions)), size)

#         states = np.array([])
#         actions = np.array([])
#         rewards = np.array([])
#         states_ = np.array([])
#         dones = np.array([])
        states = []
        actions = []
        rewards = []
        states_ = []
        dones = []
        for i in samples:
#             states = np.append(states, self.states[i])
#             actions = np.append(actions, self.actions[i])
#             rewards = np.append(rewards, self.rewards[i])
#             states_ = np.append(states_, self.states_[i])
#             dones = np.append(dones, self.dones[i])
            
            states.append(self.states[i])
            actions.append(self.actions[i])
            rewards.append(self.rewards[i])
            states_.append(self.states_[i])
            dones.append(self.dones[i])
                
        return states, actions, rewards, states_, dones

In [66]:
def updateTargetGraph(tfVars,tau):
    total_vars = len(tfVars)
    op_holder = []
    for idx,var in enumerate(tfVars[:int(total_vars/2)]):
        op_holder.append(tfVars[idx+int(total_vars/2)].assign((var.value()*tau) + ((1-tau)*tfVars[idx+int(total_vars/2)].value())))
    return op_holder

def updateTarget(op_holder,sess):
    for op in op_holder:
        sess.run(op)

In [67]:
gamma = 0.99

def discount_rewards(r):
    """Take 1D float array of rewards and compute discounted reward.
    
    Discounts rewards for a given episode.
    This is the Monte-Carlo method since we apply it to all rewards
    in a given episode.
    
    Provides more robust reward signal to DQN.
    """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [68]:
# number of experiences to use for each training step
batch_size = 32

# how often to execute training step
update_freq = 4

# discount factor on target Q-values
y = 0.99

# starting chance of random action
startE = 1

# final chance of random action
endE = 0.1

# how many training steps required to fully reduce startE to endE
anneling_steps = 20000

# number of episodes of env to train network with
num_episodes = 10000

# number of random actions before training begins
pre_train_steps = 10000

# maximum length of our episode

# Rate to update target network toward primary network
tau = 0.001 

# load saved model?
load_model = False

# path to save model to
path = "./dqn/gridWorld"

# size of final convolutional layer before
# splitting it into Advantage and Value streams
h_size = 512

In [69]:
frameShape = (84, 84, 3)
frameShape

(84, 84, 3)

In [ ]:
tf.reset_default_graph()
mainQN = Qnetwork(h_size, frameShape, batch_size)
targetQN = Qnetwork(h_size, frameShape, batch_size)

init = tf.initialize_all_variables()

saver = tf.train.Saver()

trainables = tf.trainable_variables()

targetOps = updateTargetGraph(trainables,tau)

myBuffer = experience_buffer()

# set rate of random action decrease
e = startE
stepDrop = (startE - endE) / anneling_steps

# create lists to contain total rewards and steps per episode
# movesList = []
lastReward = 0
total_steps = 0

# make path for model to be saved in
if not os.path.exists(path):
    os.makedirs(path)


with tf.Session() as sess:
    if load_model == True:
        print("Loading Model...")
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    sess.run(init)

    # set target network to be equal to primary network
    updateTarget(targetOps, sess)

    try:
        for i in range(num_episodes):
            episodeBuffer = experience_buffer()

            # reset environment and get first new observation
            env.reset()
            state = env.renderEnv()
            done = False
            rewardAll = 0
            numMoves = 0

            # the Q-Network
            while numMoves < 200:
                numMoves += 1

                # choose action with probability e of being a random action
                if np.random.rand(1) < e or total_steps < pre_train_steps:
                    action = np.random.randint(0, env.actions) # 2 = num different actions
                else:
                    action = sess.run(mainQN.predict, feed_dict={mainQN.rgb_array: [state]})

                observation, reward, done = env.step(action)
#                 if i % 100 == 0 and i != 0:
#                     env.render()
                state1 = env.renderEnv()
                total_steps += 1

                # save experience to episode buffer
                episodeBuffer.add([state], [action], [reward], [state1], [done])

                if total_steps > pre_train_steps:
                    if e > endE:
                        e -= stepDrop

                    if total_steps % update_freq == 0:
                        # random sample of experiences
                        states_t, actions_t, rewards_t, state1_t, dones_t = myBuffer.sample(batch_size)
                        

#                         states_t = np.reshape(states_t, [-1, 210, 160, 1])

                        # Double-DQN update to the target Q-values
                        Q1 = sess.run(mainQN.predict, feed_dict={
                                mainQN.rgb_array: state1_t})

                        Q2 = sess.run(targetQN.Qout, feed_dict={
                                targetQN.rgb_array: state1_t})
                        
                        # If resulting state is DONE, Q-Target = r
                        # If True: 0. If False: 1.
                        end_multiplier = -(np.array(dones_t) - 1)

                        # The Q values for predicted actions
                        doubleQ = np.array([Q2[i, j] for i, j in zip(range(len(Q1)), Q1)])
                        
                        targetQ = np.array(rewards_t) + (y * doubleQ * end_multiplier)
                        
                        # update network with target values
                        _  = sess.run(mainQN.updateModel,
                                     feed_dict={mainQN.rgb_array: states_t,
                                               mainQN.targetQ: targetQ,
                                               mainQN.actions: actions_t})
                        
                        updateTarget(targetOps, sess)

                rewardAll += reward
                state = state1

                if done == True:
                    break

            # get all experiences from this episode
            episodeRewards = np.array(episodeBuffer.rewards)

            # discount all rewards
            discountRewards = discount_rewards(episodeRewards)
            episodeBuffer.rewards = discountRewards

            # add discounted experiences to our experience buffer
            # state, action, reward, state1, done
            myBuffer.add(episodeBuffer.states,
                         episodeBuffer.actions,
                         episodeBuffer.rewards,
                         episodeBuffer.states_,
                         episodeBuffer.dones)

            lastReward = rewardAll

            # periodically save model
            if i % 100 == 0:
                saver.save(sess, path+'/model-gridWorld-'+str(i)+'.cptk')
                print("Saved Model model-gridWorld-" +str(i))
                print(i, total_steps, rewardAll, e)
                

        saver.save(sess, path+'/model-gridWorld-'+str(i)+'.cptk')
    except Exception as e:
        # if frames are still rendering, stop it
#         if env.viewer is not None:
#             env.viewer.close()
#             env.viewer = None
        
        saver.save(sess, path+'/model-gridWorld-'+str(i)+'.cptk')
        print("Saved Model model-gridWorld-" +str(i))
        raise e

print("Reward of last episode: " + str(lastReward))

Saved Model model-gridWorld-0
0 4 -1.2000000000000002 1
Saved Model model-gridWorld-100
100 676 0.7 1
Saved Model model-gridWorld-200
200 1342 0.34999999999999987 1
Saved Model model-gridWorld-300
300 2268 0.8999999999999999 1
Saved Model model-gridWorld-400
400 3088 -0.3500000000000003 1
Saved Model model-gridWorld-500
500 3765 -3.1500000000000004 1
Saved Model model-gridWorld-600
600 4504 -1.1500000000000001 1
Saved Model model-gridWorld-700
700 5275 0.29999999999999993 1
Saved Model model-gridWorld-800
800 5934 -1.25 1
Saved Model model-gridWorld-900
900 6735 -0.8500000000000001 1
Saved Model model-gridWorld-1000
1000 7395 -2.75 1
Saved Model model-gridWorld-1100
1100 8119 -1.1 1
Saved Model model-gridWorld-1200
1200 8715 -1.3 1
Saved Model model-gridWorld-1300
1300 9575 -1.25 1
Saved Model model-gridWorld-1400
1400 10224 0.5999999999999999 0.9899200000000086
Saved Model model-gridWorld-1500
1500 11039 -1.1 0.9532450000000398
Saved Model model-gridWorld-1600
1600 12030 0.04999999999

In [ ]:
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(path)
    saver.restore(sess, ckpt.model_checkpoint_path)
    sess.run(init)
    for i in range(20):
        complete = False
        iter = 0
        env = gameEnv(partial=False,size=5)
        plt.imshow(env.a)
        time.sleep(2)
        while((complete == False) & (iter < 20)):
            action = sess.run(mainQN.predict, feed_dict={mainQN.rgb_array: [env.a]})
            observation, reward, done = env.step(action)
            plt.imshow(env.a)
#             display.clear_output(wait=True)
#             display.display(plt.gcf())
            
            print(iter)
            print("reward: ", reward)
            iter += 1
            if(done):
                complete = True
                print("Complete")
            time.sleep(.5)
            plt.show()

In [43]:
ckpt.model_checkpoint_path

'./dqn/gridWorld/model-gridWorld-3825.cptk'